Flatten the Data
1. Flatten the data per item array as needed
2. Properties needed
- Tracking number
- Payment type (Prepaid/COD)
- Pickup Date Time
- Delivery Date Time
- Out for Delivery datetime(s)
- Shipment weight
- Pickup Pincode, City, State
- Drop Pincode, City, State
Transform the data
Compute the following inferred field for every shipment:
- All date times readable format in IST
- Days taken for journey completion (Pickup to Delivery, in number of days)
- Number of delivery attempts (Number of times it has been Out for Delivery + Delivered;
handle special case where Out For Delivery and Delivered happens on same day)

In [2]:

import json
import pandas as pd
from datetime import datetime
import pytz

# Load JSON data
with open("Swift Assignment 4 - Dataset.json", "r") as f:
    raw_data = json.load(f)


# Convert ISO string to IST formatted string
def to_ist_datetime(iso_str):
    utc = datetime.fromisoformat(iso_str.replace("Z", "+00:00"))
    ist = utc.astimezone(pytz.timezone("Asia/Kolkata"))
    return ist, ist.strftime('%Y-%m-%d %H:%M:%S')

# Process records
processed_data = []
for record in raw_data:
    for detail in record["trackDetails"]:
        tracking_number = detail.get("trackingNumber")
        shipment_weight = detail.get("shipmentWeight", {}).get("value", None)
        special_handlings = detail.get("specialHandlings", [])
        payment_type = "Prepaid"
        for handling in special_handlings:
            if handling["type"] == "COD":
                payment_type = "COD"
                break

        dates_dict = {d["type"]: d["dateOrTimestamp"] for d in detail.get("datesOrTimes", [])}
        pickup_dt_ist_obj, pickup_dt_ist = to_ist_datetime(dates_dict.get("ACTUAL_PICKUP"))
        delivery_dt_ist_obj, delivery_dt_ist = to_ist_datetime(dates_dict.get("ACTUAL_DELIVERY"))
        days_for_delivery = (delivery_dt_ist_obj - pickup_dt_ist_obj).days

        events = detail.get("events", [])
        delivery_dates = set()
        for event in events:
            if event["eventType"] in ["OD", "DL"]:
                ts = int(event["timestamp"]["$numberLong"]) / 1000.0
                date = datetime.fromtimestamp(ts, pytz.timezone("Asia/Kolkata")).date()
                delivery_dates.add(date)
        delivery_attempts = len(delivery_dates)

        pickup_address = detail.get("shipperAddress", {})
        pickup_city = pickup_address.get("city")
        pickup_state = pickup_address.get("stateOrProvinceCode")
        pickup_pincode = None
        for e in events:
            if e["eventType"] == "PU":
                pickup_pincode = e.get("address", {}).get("postalCode")
                break

        delivery_address = detail.get("actualDeliveryAddress", {})
        drop_city = delivery_address.get("city")
        drop_state = delivery_address.get("stateOrProvinceCode")
        drop_pincode = None
        for e in events:
            if e["eventType"] == "DL":
                drop_pincode = e.get("address", {}).get("postalCode")
                break

        processed_data.append({
            "Tracking Number": tracking_number,
            "Payment Type": payment_type,
            "Pickup Date Time in IST": pickup_dt_ist,
            "Delivery Date Time in IST": delivery_dt_ist,
            "Days Taken for Delivery": days_for_delivery,
            "Shipment Weight": shipment_weight,
            "Pickup Pincode": pickup_pincode,
            "Pickup City": pickup_city,
            "Pickup State": pickup_state,
            "Drop Pincode": drop_pincode,
            "Drop City": drop_city,
            "Drop State": drop_state,
            "Number of Delivery Attempts": delivery_attempts
        })

df = pd.DataFrame(processed_data)

summary_stats = {
    "Metric": ["Days Taken for Delivery", "Number of Delivery Attempts"],
    "Mean": [df["Days Taken for Delivery"].mean(), df["Number of Delivery Attempts"].mean()],
    "Median": [df["Days Taken for Delivery"].median(), df["Number of Delivery Attempts"].median()],
    "Mode": [df["Days Taken for Delivery"].mode()[0], df["Number of Delivery Attempts"].mode()[0]]
}
summary_df = pd.DataFrame(summary_stats)

df.head()

,Tracking Number,Payment Type,Pickup Date Time in IST,Delivery Date Time in IST,Days Taken for Delivery,Shipment Weight,Pickup Pincode,Pickup City,Pickup State,Drop Pincode,Drop City,Drop State,Number of Delivery Attempts
0,391128701026,COD,2020-03-16 15:44:00,2020-03-20 13:37:00,3,14.0,560048,Bangalore,KA,122001,Gurgaon,HR,1
1,390901883808,Prepaid,2020-03-06 16:07:00,2020-03-09 19:50:00,3,14.0,560048,Bangalore,KA,560034,Bangalore,KA,1
2,391128749178,Prepaid,2020-03-16 15:44:00,2020-03-19 15:29:00,2,14.0,560048,Bangalore,KA,380028,Ahmedabad,GJ,1
3,390807986805,Prepaid,2020-03-03 16:19:00,2020-03-07 14:24:00,3,14.0,560048,Bangalore,KA,110088,New Delhi,DL,1
4,390948921190,COD,2020-03-09 15:12:00,2020-03-13 14:44:00,3,14.0,560048,Bangalore,KA,110009,Delhi,DL,1


In [1]:
import pandas as pd

In [7]:
data = pd.read_json("Swift Assignment 4 - Dataset.json")
data.shape
data.info()


(99, 6)

In [9]:
data.sample(5)

,highestSeverity,notifications,duplicateWaybill,moreData,trackDetailsCount,trackDetails
67,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."
20,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."
95,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."
71,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."
11,SUCCESS,"[{'severity': 'SUCCESS', 'source': 'trck', 'co...",False,False,0,"[{'notification': {'severity': 'SUCCESS', 'sou..."


In [10]:
data['notifications'][0]

[{'severity': 'SUCCESS',
  'source': 'trck',
  'code': '0',
  'message': 'Request was successfully processed.',
  'localizedMessage': 'Request was successfully processed.',
  'messageParameters': []}]

In [14]:
data['trackDetails'][1]

[{'notification': {'severity': 'SUCCESS',
   'source': 'trck',
   'code': '0',
   'message': 'Request was successfully processed.',
   'localizedMessage': 'Request was successfully processed.',
   'messageParameters': []},
  'trackingNumber': '390901883808',
  'trackingNumberUniqueIdentifier': '2458915000~390901883808~FX',
  'statusDetail': {'creationTime': {'$numberLong': '1583712000000'},
   'code': 'DL',
   'description': 'Delivered',
   'location': {'streetLines': [],
    'city': 'Bangalore',
    'stateOrProvinceCode': 'KA',
    'countryCode': 'IN',
    'countryName': 'India',
    'residential': False},
   'ancillaryDetails': []},
  'informationNotes': [],
  'customerExceptionRequests': [],
  'carrierCode': 'FDXE',
  'operatingCompanyOrCarrierDescription': 'FedEx Express',
  'otherIdentifiers': [],
  'service': {'type': 'FEDEX_EXPRESS_SAVER',
   'description': 'FedEx Economy',
   'shortDescription': 'XS'},
  'packageWeight': {'units': 'KG', 'value': 14},
  'shipmentWeight': {'units

In [1]:
import json

In [2]:
with open("Swift Assignment 4 - Dataset.json", "r") as f:
    df = json.load(f)

In [8]:
df[2]

{'highestSeverity': 'SUCCESS',
 'notifications': [{'severity': 'SUCCESS',
   'source': 'trck',
   'code': '0',
   'message': 'Request was successfully processed.',
   'localizedMessage': 'Request was successfully processed.',
   'messageParameters': []}],
 'duplicateWaybill': False,
 'moreData': False,
 'trackDetailsCount': 0,
 'trackDetails': [{'notification': {'severity': 'SUCCESS',
    'source': 'trck',
    'code': '0',
    'message': 'Request was successfully processed.',
    'localizedMessage': 'Request was successfully processed.',
    'messageParameters': []},
   'trackingNumber': '391128749178',
   'trackingNumberUniqueIdentifier': '2458925000~391128749178~FX',
   'statusDetail': {'creationTime': {'$numberLong': '1584576000000'},
    'code': 'DL',
    'description': 'Delivered',
    'location': {'streetLines': [],
     'city': 'Ahmedabad',
     'stateOrProvinceCode': 'GJ',
     'countryCode': 'IN',
     'countryName': 'India',
     'residential': False},
    'ancillaryDetails':

In [3]:
df = pd.read_csv("inventory_forecasting.csv")

In [5]:
df.to_excel("inventory_forecasting.xlsx", index = False, engine = "openpyxl")

In [4]:
pip install pandas openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.
